In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import yaml

sns.set_style("darkgrid")
palette = sns.color_palette()
pd.options.mode.copy_on_write = True
scripts_dir = os.path.abspath(os.path.join(os.getcwd(), "../scripts"))
sys.path.append(scripts_dir)
workflows_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(workflows_dir)

# Define params

- `input_file`: Path (relative to project root) to a csv file containing a table with gene names and ranking metric(s). Input files should be put in the `resources` folder.
  
- `project_name`: A string to tag output files. Results will be saved in `results/{project_name}/some_filename.{project_name}.csv`

- `metrics`: A list of strings specifying columns in the input table that are used to rank the genes.

In [ ]:
#### User-defined variables ####

input_file = "resources/Chiara/edger.lrt.lfc0.KO_WT.p1.csv"
project_name = "test2"

metrics = ['logFC', 'neg_signed_logpval']#, 'signed_LR']
libraries = ["KEGG","GO"]
tools = ["clusterProfiler","gseapy"]


# ClusterProfiler

keytype = "ENSEMBL"
organismKEGG = "hsa"

In [ ]:
#### User-defined variables ####

input_file = "resources/Liana/deg.edger.lrt.batch.unm_0.6.clean.clExc7_DL.thresh.0.2.2024-01-22-17-42.P90.p19rc.csv"
project_name = "met.Exc7_DL.P90.p19rc"

metrics = ['logFC', 'neg_signed_logpval']
libraries = ["KEGG","GO"]
tools = ["clusterProfiler","gseapy","string"]


# ClusterProfiler

keytype = "SYMBOL"
organismKEGG = "mmu"

## Create config.yaml file

In [ ]:
# Create configuration dictionary
config_data = {
    'input_file': input_file,
    'project_name': project_name,
    'metrics': metrics,
    'keytype': keytype,
    'organismKEGG': organismKEGG,
    'libraries': libraries,
    'tools': tools
}

# Write to config.yaml
config_filename = "../../config/config.yaml"
with open(config_filename, 'w') as file:
    yaml.dump(config_data, file, default_flow_style=False)

print(f"Configuration file '{config_filename}' created successfully!")

savepath = f"../../results/{project_name}/"
figpath = f"../../results/{project_name}/figures/"
!mkdir $figpath

# Inspect/modify input

This space can be used to calculate further ranking metrics that are missing in the input table. As an example, we read the output from edgeR and calculate $-\mathrm{sign}(\log_2\mathrm{FC})\times\log_{10}(p\mathrm{-value})$

In [ ]:
df = pd.read_csv(f"../../{input_file}", index_col=0)
df["neg_signed_logpval"] = -np.sign(df["logFC"]) * np.log10(df["PValue"])
df["signed_LR"] = np.sign(df["logFC"]) * df["LR"]
display(df.head())
#df.to_csv(f"../../{input_file}")

## Correlations

Inspect correlations between the two different ranking metrics (e.g. logFC and signed pvalue)

In [ ]:
rank1 = "logFC"
rank2 = "neg_signed_logpval"

fig, ax = plt.subplots(1, 2, figsize=(10,5))

### Pearson correlation

correlation = df[rank1].corr(df[rank2], method='pearson')
sns.regplot(x=df[rank1], y=df["neg_signed_logpval"], ax=ax[0], scatter_kws={'alpha':0.1}, line_kws={"color":palette[3]})
ax[0].set_title(f"Pearson: {correlation:.2f}")

ax[0].set(xlabel=rank1)
ax[0].set(ylabel="-sign(logFC)*log10(p-value)")

### Spearman rank correlation

df['rank1'] = df[rank1].rank(method='average')
df['rank2'] = df[rank2].rank(method='average')
rank_correlation = df['rank1'].corr(df['rank2'], method='spearman')

sns.regplot(x=df['rank1'] ,y=df['rank2'], ax=ax[1], scatter_kws={'alpha':0.01}, line_kws={"color":palette[3]})
ax[1].set_title(f"Spearman: {rank_correlation:.2f}")
fig.tight_layout()

ax[1].set(xlabel="logFC [Rank]")
ax[1].set(ylabel="-sign(logFC)*log10(p-value) [Rank]")

# Run Snakemake

Run the following command in project root directory:

`snakemake --use-conda --cores 1` (adjust number of cores as needed)

In [ ]:
# import subprocess

# cores = 1
# command = f"snakemake -s ../Snakefile --configfile ../../config/config.yaml --use-conda --cores {cores}"
# subprocess.run(command, shell=True, check=True)

# Inspect results

## Load results

In [ ]:
sns.set_style("darkgrid")
palette = sns.color_palette()

output_files = glob.glob(f"{savepath}/syn.*[tc]sv")
print(f"Found {len(output_files)} output files:\n",*[o+"\n" for o in output_files])

In [ ]:
summary_dict = {lib: dict() for lib in libraries}
for lib in libraries:
    summary_df = pd.read_csv(f"{savepath}/syn.combined.{lib}.{project_name}.csv", index_col=0, header=[0,1,2])
    summary_df.sort_values(by=("Combined","nan","Stouffer FDR"))
    summary_dict[lib]["summary_df"] = summary_df

summary_df = summary_dict[libraries[1]]["summary_df"]
print(libraries[0], len(summary_df))
summary_df.head()

In [ ]:
# from scripts import explore_results
# import importlib
# importlib.reload(explore_results)

from scripts.explore_results import get_sig_dict, create_intersection_depth_df

qval = 0.05

for lib in summary_dict:
    print(lib)
    summary_df = summary_dict[lib]["summary_df"]
    sig_dict = get_sig_dict(summary_df, tools, metrics, qval=qval)
    summary_dict[lib]["depth_df"] = create_intersection_depth_df(sig_dict)    

## Intersection depth

We count the number of unique analysis configuations (methods, rankings) each significant term appears in. We designate this as the *intersection depth* of a term. A depth of $N$ means that the corresponding enrichment term is significant in exactly $N$ configurations.

In [ ]:
sns.set_style("whitegrid")

fig, axes = plt.subplots(1, 2, figsize=(10,3))

for ax, lib in zip(axes,summary_dict):
    depth_df = summary_dict[lib]["depth_df"]
    h = sns.histplot(depth_df['Depth'], bins=depth_df['Depth'].max() - depth_df['Depth'].min() + 1, 
                 discrete=True, ax=ax, alpha=1)
    h.bar_label(h.containers[0])
    ax.set(title=lib, xlabel="Depth", ylabel="Terms")
    ax.set_xticks(range(depth_df['Depth'].min(), depth_df['Depth'].max() + 1))
    ax.grid(axis='x')

fig.tight_layout()
fig.savefig(f"{figpath}/intersection_depth.{project_name}.pdf")

In [ ]:
sns.set_style("whitegrid")

fig, axes = plt.subplots(1, 2, figsize=(10,5))

for ax, lib in zip(axes,summary_dict):
    summary_df = summary_dict[lib]["summary_df"]
    qv = summary_df.drop(["Combined","nan"], axis=1, level=0)
    qv = qv.xs("qvalue", level=2, axis=1)
    qv = qv.replace(np.nan,1)
    qv = qv < qval

    qqv=qv.sum().reset_index()
    qqv.index = qqv["Tool"] + "\n" + qqv["Metric"]
    qqv = qqv.drop(["Tool","Metric"], axis=1)
    qqv = qqv.sort_values(by=qqv.columns[0], ascending=False)

    b = sns.barplot(x=qqv.index, y=qqv.iloc[:, 0], ax=ax, alpha=1)
    for i in b.containers:
        b.bar_label(i,)
    ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
    ax.set(title=lib, ylabel="Terms", xlabel=None)

fig.tight_layout()
fig.savefig(f"{figpath}/number_sig_terms.{project_name}.pdf")

## Venn diagrams

In [ ]:
# TO DO
from matplotlib_venn import venn2, venn3

def plot_venn(summary_df, tools, metrics):
    n_sets = len(tools) * len(metrics)
    if n_sets not in [2,3,4]:
        raise Exception(f"Venn diagram with {n_sets} sets not supported")
    return

plot_venn(summary_df, tools[:2], metrics)

## UpSet plots

Useful for visualizing set intersection with more than 3 sets. 

https://upsetplot.readthedocs.io/en/stable/index.html

In [ ]:
from matplotlib import pyplot
from upsetplot import from_memberships
from upsetplot import UpSet

depth_df = summary_dict[lib]["depth_df"]
memberships = depth_df["Factors"]
memberships_list = [categories.split(" | ") for categories in memberships.values]
upset_ready = from_memberships(memberships_list)

pd.options.mode.copy_on_write = False
UpSet(upset_ready, subset_size="count", sort_by="cardinality").plot()
pd.options.mode.copy_on_write = True
pyplot.savefig(f"{figpath}/upset.{project_name}.pdf")

# Format STRING table

User can (optionally) manually add STRING functional scoring output tables to the results folder, and they will be combined with the output from SynEnrich. For this, STRING tables have to be formatted first.

In [ ]:
from scripts.utils import format_string_table

string_file = f"{savepath}/syn.string.logFC.{project_name}.tsv"
string_file = f"{savepath}/syn.string.neg_signed_logpval.{project_name}.tsv"
string = pd.read_csv(string_file, index_col=0, sep="\t")

for library in ["GO", "KEGG"]:
    string_formatted = format_string_table(string, library)
    display(string_formatted.head())
    string_formatted.to_csv(string_file.replace(f"{project_name}.tsv", f"{library}.{project_name}.csv"))

# Test stuff

## ClusterProfiler

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
source("../../.Rprofile")
.libPaths()

In [ ]:
%%R -i input_file
library(clusterProfiler)
library(org.Hs.eg.db)

In [ ]:
%%R

filepath <- paste0("../../",input_file)

metrics = c("neg_signed_logpval","logFC")

df <- read.csv(filepath, row.names = 1)

for (metric in metrics) {
    # Check if the metric is in the columns
    if (!(metric %in% colnames(df))) {
            if (metric == "neg_signed_logpval") {
                message(paste("Adding", metric, "to df"))
                df$neg_signed_logpval <- -sign(df$logFC) * log10(df$PValue)
            } else {
                stop(paste("Metric", metric, "not in columns!"))
            }
        }
}

head(df)

In [ ]:
%%R

run_clusterProfiler <- function(df, savepath, paramset,
                                metric, cluster, overwrite=FALSE, 
                                organism.KEGG="hsa",
                                organism.GO = org.Hs.eg.db, seed=123) 
{
  set.seed(seed)

  outfile_go <- paste0(savepath,"/cluster.gseGO.",metric,".",paramset,".csv")
  outfile_kegg <- paste0(savepath,"/cluster.gseKEGG.",metric,".",paramset,".csv")
  print(outfile_go)
  print(outfile_kegg)

  if (file.exists(outfile_go) && file.exists(outfile_kegg) && !overwrite) {
    print("Existing files not overwritte, skipping")
    return
  }

  start_time <- Sys.time()

  geneList <- df[[metric]]
  names(geneList) <- df$ENTREZID
  geneList = sort(geneList, decreasing = TRUE)

  if (!file.exists(outfile_go) || overwrite) {

    ego3 <- gseGO(geneList     = geneList,
                  OrgDb        = organism.GO,
                  ont          = "ALL", ## CC MF BP
                  minGSSize    = 10,
                  maxGSSize    = 500,
                  pvalueCutoff = 1,
                  eps = 0,
                  seed = TRUE,
                  verbose = FALSE)
    write.csv(ego3,outfile_go)
  }

  if (!file.exists(outfile_kegg) || overwrite) {

    kegg <- gseKEGG(geneList     = geneList,
                  organism        =  organism.KEGG,
                  minGSSize    = 10,
                  maxGSSize    = 500,
                  pvalueCutoff = 1,
                  eps = 0,
                  seed = TRUE,
                  verbose = FALSE)
    write.csv(kegg,outfile_kegg)
  }

  end_time <- Sys.time()
  print(end_time - start_time)
}

convert_df <- function(df, OrgDb=org.Hs.eg.db) {

  if ("ENTREZID" %in% names(df)) return(df)
  
  df$ENSEMBL <- row.names(df)
  # Convert to ENTREZ ID
  # We will lose some genes here because not all IDs will be converted

  ids<-bitr(row.names(df), fromType = "ENSEMBL", toType = "ENTREZID", OrgDb=OrgDb)
  df <- merge(df, ids, by = "ENSEMBL", all.x = TRUE)
  print(paste("Before",nrow(df)))
  df <- na.omit(df)
  print(paste("After",nrow(df)))
  return(df)
}

In [ ]:
%%R
df <- convert_df(df, OrgDb=org.Hs.eg.db)
head(df)

In [ ]:
%%R -i project_name
savepath <- paste0("../../results/",project_name)
paramset <- "test"
metric <- "neg_signed_logpval"
df <- convert_df(df, OrgDb=org.Hs.eg.db)
run_clusterProfiler(df, savepath, paramset,
                                metric, overwrite=FALSE, 
                                organism.KEGG="hsa",
                                organism.GO = org.Hs.eg.db) 